<a href="https://colab.research.google.com/github/saikale/Deck/blob/main/StartupFunding_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import tensorflow as tf

from sklearn.metrics import r2_score

In [ ]:
data = pd.read_csv('/content/startup_funding.csv')
data


,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN
...,...,...,...,...,...,...,...,...,...,...
3039,3040,29/01/2015,Printvenue,NaN,NaN,NaN,Asia Pacific Internet Group,Private Equity,"45,00,000",NaN
3040,3041,29/01/2015,Graphene,NaN,NaN,NaN,KARSEMVEN Fund,Private Equity,"8,25,000",Govt backed VC Fund
3041,3042,30/01/2015,Mad Street Den,NaN,NaN,NaN,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000",NaN
3042,3043,30/01/2015,Simplotel,NaN,NaN,NaN,MakeMyTrip,Private Equity,NaN,"Strategic Funding, Minority stake"


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Sr No              3044 non-null   int64 
 1   Date dd/mm/yyyy    3044 non-null   object
 2   Startup Name       3044 non-null   object
 3   Industry Vertical  2873 non-null   object
 4   SubVertical        2108 non-null   object
 5   City  Location     2864 non-null   object
 6   Investors Name     3020 non-null   object
 7   InvestmentnType    3040 non-null   object
 8   Amount in USD      2084 non-null   object
 9   Remarks            419 non-null    object
dtypes: int64(1), object(9)
memory usage: 237.9+ KB


In [ ]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop ID and high-cardinality columns
    df = df.drop(['Sr No', 'Startup Name', 'SubVertical', 'Investors Name'], axis=1)
    
    # Clean \\xc2\\xa0 examples
    df = df.applymap(lambda x: x.replace(r'\\xc2\\xa0', '') if type(x) == str else x)
    
    # Clean target column
    df['Amount in USD'] = df['Amount in USD'].apply(lambda x: x.replace(',', '') if str(x) != 'nan' else x)
    df['Amount in USD'] = df['Amount in USD'].replace({
        'undisclosed': np.NaN,
        'unknown': np.NaN,
        'Undisclosed': np.NaN,
        'N/A': np.NaN,
        '14342000+': '14342000'
    })
    
    # Drop missing target rows
    missing_target_rows = df[df['Amount in USD'].isna()].index
    df = df.drop(missing_target_rows, axis=0).reset_index(drop=True)
    
    # Drop columns with more than 25% missing values
    df = df.drop('Remarks', axis=1)
    
    # Fill categorical missing values with most frequent occurence
    for column in ['Industry Vertical', 'City  Location', 'InvestmentnType']:
        df[column] = df[column].fillna(df[column].mode()[0])
    
    # Clean date column
    df['Date dd/mm/yyyy'] = df['Date dd/mm/yyyy'].replace({
        '05/072018': '05/07/2018',
        '01/07/015': '01/07/2015',
        '22/01//2015': '22/01/2015'
    })
    
    # Extract date features
    df['Date dd/mm/yyyy'] = pd.to_datetime(df['Date dd/mm/yyyy'])
    df['Year'] = df['Date dd/mm/yyyy'].apply(lambda x: x.year)
    df['Month'] = df['Date dd/mm/yyyy'].apply(lambda x: x.month)
    df['Day'] = df['Date dd/mm/yyyy'].apply(lambda x: x.day)
    df = df.drop('Date dd/mm/yyyy', axis=1)
    
    # Convert target column to float
    df['Amount in USD'] = df['Amount in USD'].astype(np.float)
    
    # Split df into X and y
    y = df['Amount in USD']
    X = df.drop('Amount in USD', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [ ]:
X_train

,Industry Vertical,City Location,InvestmentnType,Year,Month,Day
924,eCommerce,Noida,Private Equity,2016,4,10
1108,Consumer Internet,Bangalore,Seed Funding,2016,7,21
1059,Consumer Internet,Mumbai,Private Equity,2016,8,29
160,Consumer Internet,Bengaluru,Seed/ Angel Funding,2018,8,8
1696,on-demand delivery service,Gurgaon,Seed Funding,2015,8,17
...,...,...,...,...,...,...
960,eCommerce,Ahmedabad,Private Equity,2016,10,26
905,Consumer Internet,Mumbai,Private Equity,2016,11,24
1096,eCommerce,New Delhi,Seed Funding,2016,7,15
235,Finance,Chennai,Seed / Angel Funding,2018,2,5


In [ ]:
y_train

924     4200000.0
1108     595000.0
1059    3000000.0
160     4000000.0
1696     310000.0
          ...    
960     1000000.0
905     4000000.0
1096     250000.0
235      450000.0
1061    1000000.0
Name: Amount in USD, Length: 1451, dtype: float64

In [ ]:
def build_model():
    inputs = tf.keras.Input(shape=(532,))
    x = tf.keras.layers.Dense(128, activation='relu')(inputs)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='linear')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer='adam',
        loss='mse'
    )
    
    return model

In [ ]:
nominal_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('nominal', nominal_transformer, ['Industry Vertical', 'City  Location', 'InvestmentnType'])
], remainder='passthrough')

regressor = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('regressor', regressor)
])

In [ ]:
model.fit(
    X_train,
    y_train,
    regressor__validation_split=0.2,
    regressor__batch_size=32,
    regressor__epochs=100,
    regressor__callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
37/37 [==============================] - 1s 6ms/step - loss: 17648144099573760.0000 - val_loss: 29475037702520832.0000
Epoch 2/100
37/37 [==============================] - 0s 3ms/step - loss: 17648135509639168.0000 - val_loss: 29475037702520832.0000
Epoch 3/100
37/37 [==============================] - 0s 3ms/step - loss: 17648130140930048.0000 - val_loss: 29475029112586240.0000
Epoch 4/100
37/37 [==============================] - 0s 3ms/step - loss: 17648124772220928.0000 - val_loss: 29475020522651648.0000
Epoch 5/100
37/37 [==============================] - 0s 3ms/step - loss: 17648107592351744.0000 - val_loss: 29475001195298816.0000
Epoch 6/100
37/37 [==============================] - 0s 3ms/step - loss: 17648088264998912.0000 - val_loss: 29474964688076800.0000
Epoch 7/100
37/37 [==============================] - 0s 3ms/step - loss: 17648046389067776.0000 - val_loss: 29474921738403840.0000
Epoch 8/100
37/37 [==============================] - 0s 3ms/step - loss: 1764800129

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('nominal',
                                                  Pipeline(memory=None,
                                                           steps=[('onehot',
                                                                   OneHotEncoder(categories='auto',
                                                                                 drop=None,
                                                                                 dtype=<class 'numpy.float64'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))],
                                    

In [ ]:
y_pred = model.predict(X_test)

rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print("     Test RMSE: {:.2f}".format(rmse))

r2 = r2_score(y_test, y_pred)
print("Test R^2 Score: {:.5f}".format(r2))

     Test RMSE: 56537453.92
Test R^2 Score: -0.02713
